Superheroes and sticks
============

Some time ago, when I was around 17 years old, I have started solving physics problems from the book "Physical Problems" edition 1981 written by Savchenko O. Y. I was solving them one by one consecutively. But I did not finish them at the time... 


I still have the book. So, last year, after I broke a leg at a soccer game, I had a bit more time, when I could only read, think or watch television. At that point, I had an idea to resume the problem-solving activity, and still solve some problems occasionally, even though my leg is OK now.

Recently, I was talking with a friend about one of the problems during lunch. That one was about a satellite that was losing height due to air friction (this one is interesting to me since the speed of the satellite is actually increasing). The friend suggested that I could write up my solutions in Latex. At that point, I did not feel like any of the solutions I have come up with was good enough to spend time writing them up. But today, I think, I have a good one. I would like to share it here. Who knows, maybe someone will point me to an even easier and nicer solution.

Here is my interpretation of the problem statement (it is Problem 5.42 in my edition of the book):

> Superman and Ironman throw heavy sticks upwards at a pole of the Earth. The Superman's stick fell back after 30 days and the Ironman's stick fell back after 7 days. What is the difference between the initial speeds of the sticks?


**Solution**
It is probably possible to verify that the sticks would go far enough that the approximation $\vec{g}=const$ is not valid anymore. Although I did not to it myself. I have assumed from the start that it is not valid for the given travel times.


From the energy conservation law we can obtain the expression for the speed of a stick, for example, on its way up at a distance $r$ from the Earth center:

$$
    e=\frac{E}{m}=\frac{v^2}{2} - \frac{GM}{r}
$$

Then, by equating it to the energy at the highest point, we get:

$$
e=\frac{v^2}{2} - \frac{gR^2}{r}=-\frac{gR^2}{x}
$$

$x$ is the distance from the center of the Earth to the highest of the trajectory of a stick. Here I have used the definition of the free-fall acceleration at the Earth surface:

$$
g = \frac{GM}{r^2}
$$

$G$ is the gravitational constant, $R$ is the Earth radius.

Eliminating the speed we get

$$
v = \sqrt{\frac{2gR^2}{x}}\cdot\sqrt{\frac{x}{R} - 1}
$$

On the way up $v=\frac{dr}{dt}$, so:

$$
\frac{dr}{dt} = \sqrt{\gamma}\cdot\sqrt{\frac{x}{R} - 1}\\
\gamma\equiv\frac{2gR^2}{x}
$$

I have integrated the above equation by using the following substitution:
$$
\frac{1}{\cos^2{\alpha}} = \frac{x}{R}
$$

and obtained the time ($t$) it takes the stick to travel one way as a function of the maximum distance from the centre of the Earth ($x$):

$$
t = \frac{1}{\sqrt{\gamma}}\left( x \cos^{-1}\left(\frac{R}{x}\right) + \sqrt{R(x-R)} \right)
$$

I would prefer to have it the other way around... 

The fact that the stick speed should be bounded between the first $v_1=\sqrt{gR}$ and the second $v_2=\sqrt{2gR}$ cosmic speeds inspired me to use a search algorithm to solve for $x$. To include the speed at the Earth surface I have used the expressions for the total energy at the surface and at the highest point to construct the following system of equations:

$$
e_0 = \frac{v_0^2}{2} - gR \\
x = -\frac{gR^2}{e_0}\\
t = \frac{1}{\sqrt{\gamma}}\left( x \cos^{-1}\left(\frac{R}{x}\right) + \sqrt{R(x-R)} \right) \\
\gamma\equiv\frac{2gR^2}{x}
$$

Finally, I used Python to implement an iterative search algorithm to find solutions of the system above.
At the start, the limits of the speed are set to:

$$
vmin_0 = \sqrt{gR} \\
vmax_0 = \sqrt{2gR}
$$

then we substitute $v_n = (vmin_n + vmax_n) / 2$ until the calculated $2t_n$ is close enough to the travel times given in the problem statement. Until this happens the following iterations are perfomed (based on the fact that $v$ grows monotonuously with $t$):

if $2t_n > 2t$, then $vmax_{n + 1} = v_n$, else $vmin_{n+1} = v_n$ and the next trial for $v$ is calculated as:

$$
v_{n+1} = (vmin_{n+1} + vmax_{n + 1}) / 2
$$


Below is the implementation of the algorithm:

In [1]:
import sys

import math

g = 9.8

r = 6400e3

# problem 5.42 from Savchenko


def e0(v):
    # total energy
    return v * v / 2 - g * r


def xm(v):
    # maximum distance from the Earth centre
    return -g*r*r / e0(v)


def gamma(x):
    # a helper function
    return 2*g*r*r/x

def tme(v):
    # half of the travel time
    x = xm(v)
    assert x >= r
    return (1/gamma(x)**0.5) * (x*math.acos((r/x)**0.5) + (r*(x-r))**0.5)


def main(tdays=7):
    nmax = 1000
    epsmax = 1e-6

    vmax = (2*g*r) **0.5
    vmin = (g*r) ** 0.5

    t = tdays * 24 * 3600

    for i in range(nmax):
        v = 0.5 * (vmin + vmax)
        ti = tme(v)

        eps = abs(t - ti)
        if  eps <= epsmax:
            break

        if ti < t:
            vmin = v
        else:
            vmax = v

    print(v, eps)
    return v
    

print(main(tdays=30./2.) - main(tdays=7./2.))

11156.085675766404 8.069910109043121e-07
11083.833910366175 4.91156242787838e-07
72.25176540022949


In conclusion, the solution gives 11156 m/s for the Superman's stick and 11084 m/s for the Ironman's stick. Therefore the difference is around 72 m/s.